<a href="https://colab.research.google.com/github/KonradKomor/Carbios-ARIMA-stock-price-prediction/blob/master/Carbios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade Cython
! pip install statsmodels
import statsmodels.api as sm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import plotly.express as pt

In [ ]:
df= pd.read_excel('DaneCarbios.xlsx')

In [ ]:
df

,Date,Open,High,Low,Close,Volume,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,2020-08-17 17:39:59.904,26.85,28.00,26.80,27.35,74345,NaN,NaN,NaN
1,2020-08-18 17:39:59.904,27.50,29.10,27.50,29.10,71063,NaN,NaN,ALCRB
2,2020-08-19 17:39:59.904,29.50,29.65,28.25,29.50,51476,NaN,NaN,NaN
3,2020-08-20 17:39:59.904,29.00,29.90,28.45,29.60,40639,NaN,NaN,NaN
4,2020-08-21 17:39:59.904,30.00,32.45,29.95,31.40,89821,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
252,2021-08-10 17:39:59.904,43.10,43.90,42.54,43.76,15391,NaN,NaN,NaN
253,2021-08-11 17:39:59.904,43.90,44.00,43.52,43.88,13831,NaN,NaN,NaN
254,2021-08-12 17:39:59.904,43.88,45.58,43.52,45.58,37051,NaN,NaN,NaN
255,2021-08-13 17:39:59.904,45.50,45.56,43.32,44.00,22132,NaN,NaN,NaN


In [ ]:
fig=pt.area(data_frame=df, x='Date',y='Close')
fig.show()

In [ ]:
fig=pt.area(data_frame=df, x='Date',y='Close',log_y=True,title="Carbios stock price (%)")
fig.show()

In [ ]:
x=df.Close
result=adfuller(x)

In [ ]:
#Run Augmented Dickey-Fuller Test for testing the stationarity of the data
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('critical Values:')
for key, value in result[4].items():
  print('\t%s: %.3f' % (key, value))

ADF Statistic: -2.078506
p-value: 0.253245
critical Values:
	1%: -3.456
	5%: -2.873
	10%: -2.573
